In [ ]:
import numpy as np
import sklearn as sl
import pandas as pd 
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from google.colab import files
from math import sqrt
from sklearn import model_selection
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler


In [ ]:
# The following code will mount the drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Insert file names here#
data = pd.read_csv('/content/gdrive/MyDrive/DS 320 Project/Fema and Zillow Joined/fema_zillow_joined.csv')



**TAKE A PEEK**


**PREPROCESSING**

In [ ]:
'''
We see that there are some variables which won't be of use - drop these.

Variables to drop: V1, full_county, county, state.x, countyCode, stateCode, 
designatedArea, state.y, fipsCountyCode, fipsStateCode, region.x, Metro, 
region.y, RegionName
'''

data_dropped = data.drop(["V1", "full_county", "county", "state.x", "countyCode", "stateCode", 
"designatedArea", "state.y", "region.x", "Metro", 
"region.y", "RegionName"], axis = 1)
data_dropped

,county_disaster_count,countyObligated,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,Poverty,Unemployment,county_disasters_perCap,county_obligated_perCap,fipsCountyCode,fipsStateCode,count,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,...,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,SizeRank,which_measurement
0,14,446195.34,24997,12308,12689,1.2,68.9,27.9,0.1,0.0,0.0,34799,22.5,10.7,0.000560,17.849956,1,45,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,72292.0,72138.0,72013.0,70824.0,70505.0,70464.0,70946.0,70189.0,69622.0,69240.0,69792.0,69766.0,70275.0,70693.0,71476.0,72666.0,72961.0,73187.0,73240.0,74496.0,75282.0,76323.0,76720.0,78714.0,80047.0,81381.0,81885.0,82701.0,84107.0,85481.0,86823.0,87662.0,88764.0,89554.0,90511.0,90467.0,90786.0,91118.0,4045.0,City_Zhvi_1bedroom
1,14,446195.34,24997,12308,12689,1.2,68.9,27.9,0.1,0.0,0.0,34799,22.5,10.7,0.000560,17.849956,1,45,12,54183.0,54057.0,53857.0,53818.0,53472.0,53289.0,53333.0,53722.0,53946.0,53957.0,54162.0,54622.0,55075.0,55232.0,55191.0,55060.0,55244.0,55590.0,55870.0,55861.0,55829.0,...,70584.0,70431.0,70917.0,70931.0,71445.0,71926.0,72284.0,72453.0,72557.0,73032.0,73643.0,73531.0,72891.0,72747.0,72824.0,73589.0,73780.0,74449.0,74808.0,75286.0,75358.0,75549.0,75672.0,77004.0,78601.0,79572.0,79712.0,79340.0,79570.0,79565.0,79912.0,80180.0,80551.0,80592.0,80549.0,79890.0,79187.0,78692.0,4045.0,City_Zhvi_2bedroom
2,14,446195.34,24997,12308,12689,1.2,68.9,27.9,0.1,0.0,0.0,34799,22.5,10.7,0.000560,17.849956,1,45,12,35083.0,34255.0,33700.0,33536.0,33750.0,34518.0,35296.0,35757.0,35502.0,35544.0,36010.0,36658.0,37188.0,37563.0,38258.0,38809.0,39324.0,39516.0,39741.0,40005.0,40467.0,...,62262.0,62840.0,63449.0,63754.0,64202.0,64241.0,64514.0,64909.0,65793.0,66586.0,67320.0,67685.0,67881.0,67995.0,67941.0,68364.0,68585.0,68937.0,68978.0,69246.0,69225.0,69330.0,69417.0,69953.0,70232.0,70398.0,70639.0,70708.0,70972.0,71407.0,72028.0,72446.0,72830.0,73123.0,73536.0,73754.0,74285.0,74958.0,9525.0,City_Zhvi_2bedroom
3,14,446195.34,24997,12308,12689,1.2,68.9,27.9,0.1,0.0,0.0,34799,22.5,10.7,0.000560,17.849956,1,45,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,41782.0,41725.0,41493.0,41038.0,40820.0,40815.0,41059.0,41033.0,41036.0,41141.0,41687.0,42050.0,42530.0,43054.0,43890.0,44849.0,45558.0,46100.0,46183.0,46393.0,46573.0,46794.0,47078.0,47180.0,47518.0,47769.0,48134.0,48203.0,48285.0,48366.0,48624.0,48801.0,48898.0,48837.0,48494.0,48273.0,47767.0,47251.0,12741.0,City_Zhvi_2bedroom
4,14,446195.34,24997,12308,12689,1.2,68.9,27.9,0.1,0.0,0.0,34799,22.5,10.7,0.000560,17.849956,1,45,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,66397.0,66992.0,67722.0,68035.0,67876.0,67889.0,67731.0,67832.0,67978.0,68282.0,68546.0,68888.0,69146.0,69107.0,69079.0,69355.0,70258.0,70393.0,70611.0,70470.0,70513.0,70546.0,70797.0,71484.0,71886.0,72313.0,72709.0,73217.0,73221.0,73710.0,74165.0,74749.0,75156.0,75460.0,75579.0,75335.0,75282.0,75256.0,20429.0,City_Zhvi_2bedroom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159166,159,3919333.57,12060,5978,6082,93.0,6.4,0.4,0.1,0.0,0.0,26672,33.4,12.2,0.013184,324.986200,507,48,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
'''
There are multiple measurements being used for house prices. 
Let's use a general measurement: City_Zhvi_AllHomes
'''
is_AllHomes = data_dropped["which_measurement"] == "City_Zhvi_AllHomes"
data_filtered = data_dropped[is_AllHomes]
data_filtered = data_filtered.drop("which_measurement", axis = 1)
data_filtered

,county_disaster_count,countyObligated,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,Poverty,Unemployment,county_disasters_perCap,county_obligated_perCap,fipsCountyCode,fipsStateCode,count,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,...,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,SizeRank
15,14,446195.34,24997,12308,12689,1.2,68.9,27.9,0.1,0.0,0.0,34799,22.5,10.7,0.000560,17.849956,1,45,12,73914.0,74403.0,74573.0,74319.0,74154.0,74079.0,74322.0,74582.0,74899.0,75060.0,75343.0,75794.0,76460.0,76869.0,77340.0,77514.0,77684.0,77937.0,78344.0,78765.0,79133.0,...,115206.0,115425.0,115637.0,116094.0,116395.0,116915.0,117612.0,118456.0,119013.0,119660.0,120437.0,121480.0,122519.0,123340.0,123962.0,124367.0,124889.0,125386.0,126249.0,126608.0,127343.0,127282.0,127779.0,127844.0,128782.0,129649.0,130532.0,130991.0,131506.0,132121.0,132491.0,133190.0,133661.0,134533.0,134923.0,135466.0,135279.0,135228.0,135226.0,4045.0
16,14,446195.34,24997,12308,12689,1.2,68.9,27.9,0.1,0.0,0.0,34799,22.5,10.7,0.000560,17.849956,1,45,12,55182.0,54318.0,53243.0,52132.0,51267.0,50866.0,50882.0,51105.0,51043.0,51036.0,51338.0,51952.0,52504.0,53160.0,53792.0,54595.0,55084.0,55157.0,55348.0,55514.0,56031.0,...,86257.0,87038.0,87780.0,88106.0,88384.0,88623.0,88920.0,89268.0,89719.0,90435.0,91255.0,92166.0,92213.0,92144.0,92651.0,93612.0,94642.0,95037.0,95584.0,95970.0,96798.0,97242.0,97835.0,98099.0,98872.0,99383.0,99471.0,99477.0,99577.0,100124.0,100589.0,101222.0,101479.0,101919.0,102169.0,102665.0,103073.0,103801.0,104604.0,9525.0
17,14,446195.34,24997,12308,12689,1.2,68.9,27.9,0.1,0.0,0.0,34799,22.5,10.7,0.000560,17.849956,1,45,12,32110.0,32222.0,32474.0,32902.0,33248.0,33390.0,33317.0,33252.0,33259.0,33302.0,33407.0,33536.0,33760.0,33940.0,33691.0,33284.0,32963.0,33214.0,33396.0,33599.0,33494.0,...,36710.0,36889.0,37065.0,37218.0,37422.0,38143.0,38622.0,39200.0,39238.0,39895.0,40221.0,40809.0,41034.0,41157.0,41687.0,42326.0,43151.0,43328.0,43683.0,44039.0,44974.0,45735.0,46265.0,46585.0,47228.0,48472.0,49212.0,49630.0,49760.0,50340.0,50874.0,51361.0,51621.0,51773.0,52253.0,52777.0,53129.0,53010.0,53032.0,12741.0
18,14,446195.34,24997,12308,12689,1.2,68.9,27.9,0.1,0.0,0.0,34799,22.5,10.7,0.000560,17.849956,1,45,12,65153.0,65293.0,65090.0,64229.0,63317.0,63168.0,63907.0,64653.0,65045.0,65003.0,65133.0,65545.0,66155.0,66706.0,67696.0,68614.0,68887.0,68322.0,68397.0,68832.0,69411.0,...,117285.0,118021.0,118829.0,119566.0,119915.0,120365.0,121069.0,122100.0,122952.0,123954.0,125119.0,126389.0,126825.0,126832.0,126748.0,127198.0,127944.0,128338.0,129258.0,129454.0,130838.0,131048.0,131846.0,131869.0,132919.0,133917.0,134866.0,135355.0,136008.0,136871.0,137374.0,138322.0,138399.0,139194.0,139446.0,140196.0,140397.0,140818.0,141281.0,20429.0
19,14,446195.34,24997,12308,12689,1.2,68.9,27.9,0.1,0.0,0.0,34799,22.5,10.7,0.000560,17.849956,1,45,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,74414.0,75542.0,75945.0,76513.0,76106.0,76135.0,75439.0,76133.0,76526.0,76937.0,76814.0,77008.0,77458.0,77398.0,77818.0,79010.0,80397.0,82031.0,83507.0,84293.0,84989.0,84717.0,85078.0,84840.0,85063.0,85921.0,86499.0,86362.0,86128.0,85577.0,85339.0,85031.0,84764.0,85050.0,85005.0,85139.0,84739.0,83951.0,83164.0,22324.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
'''
Standardize
'''
#Import the scaler to beging standardization
prices = pd.DataFrame(data_filtered["2020-03"]) # Save output variable.
train = data_filtered.drop("2020-03", axis = 1)

#test = test.drop('id',axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#temp_norm = data_filtered.append(test)
#temp_norm = temp_norm.drop('Time', axis =1)

col_names = train.columns

#Apply the scaler to the merged data set
min_max_scaler = preprocessing.MinMaxScaler()
train_std = min_max_scaler.fit_transform(train)
train_std = pd.DataFrame(train_std, columns=col_names)

train_std

,county_disaster_count,countyObligated,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,Poverty,Unemployment,county_disasters_perCap,county_obligated_perCap,fipsCountyCode,fipsStateCode,count,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,...,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,SizeRank
0,0.002088,0.000221,0.002415,0.002415,0.002422,0.012158,0.689655,0.324796,0.001295,0.0,0.0,0.148581,0.444915,0.380228,0.003999,0.000766,0.001299,0.800000,0.370370,0.010056,0.009912,0.009655,0.009427,0.009329,0.009262,0.009324,0.009303,0.009317,0.009244,0.009187,0.009236,0.009252,0.009251,0.009190,0.009154,0.009041,0.008911,0.008732,0.008622,0.008518,...,0.008053,0.008004,0.007888,0.007743,0.007600,0.007514,0.007459,0.007450,0.007408,0.007426,0.007476,0.007613,0.007726,0.007862,0.007312,0.007425,0.007594,0.007628,0.007661,0.007636,0.007684,0.007725,0.007717,0.007751,0.007824,0.008010,0.008159,0.008288,0.008335,0.008499,0.008685,0.008909,0.009074,0.009185,0.009287,0.009295,0.009278,0.009155,0.009068,0.137356
1,0.002088,0.000221,0.002415,0.002415,0.002422,0.012158,0.689655,0.324796,0.001295,0.0,0.0,0.148581,0.444915,0.380228,0.003999,0.000766,0.001299,0.800000,0.370370,0.007508,0.007236,0.006894,0.006613,0.006450,0.006360,0.006383,0.006375,0.006350,0.006286,0.006260,0.006331,0.006353,0.006398,0.006392,0.006447,0.006411,0.006306,0.006169,0.006077,0.006031,...,0.005983,0.005993,0.005948,0.005878,0.005768,0.005705,0.005654,0.005632,0.005582,0.005598,0.005650,0.005769,0.005862,0.005917,0.005286,0.005370,0.005536,0.005603,0.005630,0.005605,0.005648,0.005695,0.005720,0.005760,0.005830,0.005974,0.006076,0.006132,0.006144,0.006244,0.006387,0.006565,0.006695,0.006768,0.006827,0.006827,0.006818,0.006764,0.006753,0.323441
2,0.002088,0.000221,0.002415,0.002415,0.002422,0.012158,0.689655,0.324796,0.001295,0.0,0.0,0.148581,0.444915,0.380228,0.003999,0.000766,0.001299,0.800000,0.370370,0.004369,0.004293,0.004205,0.004173,0.004183,0.004175,0.004180,0.004148,0.004137,0.004101,0.004073,0.004087,0.004085,0.004085,0.004003,0.003931,0.003836,0.003797,0.003722,0.003678,0.003605,...,0.002562,0.002550,0.002521,0.002482,0.002436,0.002416,0.002433,0.002446,0.002451,0.002448,0.002492,0.002543,0.002595,0.002633,0.001976,0.002027,0.002104,0.002156,0.002168,0.002167,0.002196,0.002252,0.002297,0.002331,0.002376,0.002459,0.002572,0.002643,0.002677,0.002727,0.002812,0.002913,0.002984,0.003024,0.003045,0.003067,0.003077,0.003057,0.003012,0.432646
3,0.002088,0.000221,0.002415,0.002415,0.002422,0.012158,0.689655,0.324796,0.001295,0.0,0.0,0.148581,0.444915,0.380228,0.003999,0.000766,0.001299,0.800000,0.370370,0.008864,0.008699,0.008428,0.008147,0.007966,0.007898,0.008017,0.008065,0.008091,0.008006,0.007942,0.007987,0.008005,0.008028,0.008044,0.008103,0.008018,0.007811,0.007623,0.007535,0.007471,...,0.008164,0.008148,0.008065,0.007957,0.007827,0.007741,0.007679,0.007669,0.007635,0.007671,0.007744,0.007909,0.008038,0.008139,0.007538,0.007607,0.007783,0.007832,0.007859,0.007835,0.007873,0.007957,0.007967,0.008021,0.008094,0.008291,0.008453,0.008589,0.008639,0.008817,0.009026,0.009268,0.009456,0.009541,0.009639,0.009636,0.009632,0.009535,0.009480,0.693708
4,0.002088,0.000221,0.002415,0.002415,0.002422,0.012158,0.689655,0.324796,0.001295,0.0,0.0,0.148581,0.444915,0.380228,0.003999,0.000766,0.001299,0.800000,0.370370,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005165,0.005170,0.005162,0.005085,0.005009,0.004913,0.004857,0.004779,0.00

In [ ]:
'''
Split into training, validation, and test sets. 
80/10/10
'''
from sklearn.model_selection import train_test_split

train_size=0.8

X = train_std
y = prices

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

**PREDICTION**

**LINEAR REGRESSION ON ENTIRE DATASET**

In [ ]:
# Make Linear Regression Model 
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
pred = model.predict(X_valid)
print(pred)


[[134962.1938768 ]
 [116527.46993656]
 [255308.94547623]
 ...
 [ 87840.54885427]
 [196909.69022284]
 [187148.59291803]]


MEASURE PERFORMANCE USING RMSE

In [ ]:
mse = mean_squared_error(y_valid, pred)
rmse = sqrt(mse)
print(rmse)

359.33284079503517


**LINEAR REGRESSION ON JUST ZILLOW DATASET**

In [ ]:
train_std

,county_disaster_count,countyObligated,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,Poverty,Unemployment,county_disasters_perCap,county_obligated_perCap,fipsCountyCode,fipsStateCode,count,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,2000-11,2000-12,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,...,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,SizeRank
0,0.002088,0.000221,0.002415,0.002415,0.002422,0.012158,0.689655,0.324796,0.001295,0.0,0.0,0.148581,0.444915,0.380228,0.003999,0.000766,0.001299,0.800000,0.370370,0.010056,0.009912,0.009655,0.009427,0.009329,0.009262,0.009324,0.009303,0.009317,0.009244,0.009187,0.009236,0.009252,0.009251,0.009190,0.009154,0.009041,0.008911,0.008732,0.008622,0.008518,...,0.008053,0.008004,0.007888,0.007743,0.007600,0.007514,0.007459,0.007450,0.007408,0.007426,0.007476,0.007613,0.007726,0.007862,0.007312,0.007425,0.007594,0.007628,0.007661,0.007636,0.007684,0.007725,0.007717,0.007751,0.007824,0.008010,0.008159,0.008288,0.008335,0.008499,0.008685,0.008909,0.009074,0.009185,0.009287,0.009295,0.009278,0.009155,0.009068,0.137356
1,0.002088,0.000221,0.002415,0.002415,0.002422,0.012158,0.689655,0.324796,0.001295,0.0,0.0,0.148581,0.444915,0.380228,0.003999,0.000766,0.001299,0.800000,0.370370,0.007508,0.007236,0.006894,0.006613,0.006450,0.006360,0.006383,0.006375,0.006350,0.006286,0.006260,0.006331,0.006353,0.006398,0.006392,0.006447,0.006411,0.006306,0.006169,0.006077,0.006031,...,0.005983,0.005993,0.005948,0.005878,0.005768,0.005705,0.005654,0.005632,0.005582,0.005598,0.005650,0.005769,0.005862,0.005917,0.005286,0.005370,0.005536,0.005603,0.005630,0.005605,0.005648,0.005695,0.005720,0.005760,0.005830,0.005974,0.006076,0.006132,0.006144,0.006244,0.006387,0.006565,0.006695,0.006768,0.006827,0.006827,0.006818,0.006764,0.006753,0.323441
2,0.002088,0.000221,0.002415,0.002415,0.002422,0.012158,0.689655,0.324796,0.001295,0.0,0.0,0.148581,0.444915,0.380228,0.003999,0.000766,0.001299,0.800000,0.370370,0.004369,0.004293,0.004205,0.004173,0.004183,0.004175,0.004180,0.004148,0.004137,0.004101,0.004073,0.004087,0.004085,0.004085,0.004003,0.003931,0.003836,0.003797,0.003722,0.003678,0.003605,...,0.002562,0.002550,0.002521,0.002482,0.002436,0.002416,0.002433,0.002446,0.002451,0.002448,0.002492,0.002543,0.002595,0.002633,0.001976,0.002027,0.002104,0.002156,0.002168,0.002167,0.002196,0.002252,0.002297,0.002331,0.002376,0.002459,0.002572,0.002643,0.002677,0.002727,0.002812,0.002913,0.002984,0.003024,0.003045,0.003067,0.003077,0.003057,0.003012,0.432646
3,0.002088,0.000221,0.002415,0.002415,0.002422,0.012158,0.689655,0.324796,0.001295,0.0,0.0,0.148581,0.444915,0.380228,0.003999,0.000766,0.001299,0.800000,0.370370,0.008864,0.008699,0.008428,0.008147,0.007966,0.007898,0.008017,0.008065,0.008091,0.008006,0.007942,0.007987,0.008005,0.008028,0.008044,0.008103,0.008018,0.007811,0.007623,0.007535,0.007471,...,0.008164,0.008148,0.008065,0.007957,0.007827,0.007741,0.007679,0.007669,0.007635,0.007671,0.007744,0.007909,0.008038,0.008139,0.007538,0.007607,0.007783,0.007832,0.007859,0.007835,0.007873,0.007957,0.007967,0.008021,0.008094,0.008291,0.008453,0.008589,0.008639,0.008817,0.009026,0.009268,0.009456,0.009541,0.009639,0.009636,0.009632,0.009535,0.009480,0.693708
4,0.002088,0.000221,0.002415,0.002415,0.002422,0.012158,0.689655,0.324796,0.001295,0.0,0.0,0.148581,0.444915,0.380228,0.003999,0.000766,0.001299,0.800000,0.370370,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005165,0.005170,0.005162,0.005085,0.005009,0.004913,0.004857,0.004779,0.00

In [ ]:
'''
We need to separate out the Zillow dataset.
'''
'''
Split into training, validation, and test sets. 
80/10/10
'''
from sklearn.model_selection import train_test_split

train_size=0.8

X = train_std.iloc[:,19:262]
y = prices

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [ ]:
# Make Linear Regression Model 
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
pred = model.predict(X_valid)
print(pred)


[[ 88430.07283492]
 [ 57650.77459038]
 [429428.26987145]
 ...
 [270702.395809  ]
 [166959.74182253]
 [203803.17538301]]


MEASURE PERFORMANCE USING RMSE

In [ ]:
mse = mean_squared_error(y_valid, pred)
rmse = sqrt(mse)
print(rmse)

1112.708993718504


**LINEAR REGRESSION ON FEMA DATASET**

In [ ]:
'''
We need to separate out the Zillow dataset.
'''
'''
Split into training, validation, and test sets. 
80/10/10
'''
from sklearn.model_selection import train_test_split

train_size=0.8

X = train_std.iloc[:,0:18]
y = prices

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [ ]:
# Make Linear Regression Model 
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
pred = model.predict(X_valid)
print(pred)


[[134383.34413421]
 [230324.57469923]
 [159858.41775009]
 ...
 [136817.67579702]
 [176275.13715744]
 [166500.33955976]]


MEASURE PERFORMANCE USING RMSE

In [ ]:
mse = mean_squared_error(y_valid, pred)
rmse = sqrt(mse)
print(rmse)

202344.09992739235
